###  Real-Time Object Detection Dashboard with YOLOv8

In [ ]:
from flask import Flask, Response, jsonify
import cv2
from ultralytics import YOLO
import os
import time
from collections import defaultdict

In [ ]:
app = Flask(__name__)

In [ ]:
# Initialize YOLO model
model_path = "yolov8n.pt"
if not os.path.exists(model_path):
    print("Downloading model...")
    yolo = YOLO("yolov8n.pt")  # This will download if missing
else:
    try:
        yolo = YOLO(model_path)  # Attempt to load
    except RuntimeError as e:
        print(f"Error: {e}. File may be corrupted. Redownloading...")
        os.remove(model_path)  # Delete corrupted file
        yolo = YOLO("yolov8n.pt")  # Fresh download


In [ ]:
# Detection statistics
detection_stats = {
    'total_objects': 0,
    'class_counts': defaultdict(int),
    'last_update': time.time(),
    'fps': 0
}

In [ ]:
# Function to get class colors
def getColours(cls_num):
    base_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    color_index = cls_num % len(base_colors)
    increments = [(1, -2, 1), (-2, 1, -1), (1, -1, 2)]
    color = [base_colors[color_index][i] + increments[color_index][i] * 
             (cls_num // len(base_colors)) % 256 for i in range(3)]
    return tuple(color)

In [ ]:
def generate_frames():
    videoCap = cv2.VideoCapture(0)
    frame_count = 0
    start_time = time.time()
    
    while True:
        ret, frame = videoCap.read()
        if not ret:
            break

        results = yolo.track(frame, stream=True)
        current_objects = 0
        current_classes = defaultdict(int)

        for result in results:
            classes_names = result.names
            for box in result.boxes:
                if box.conf[0] > 0.4:
                    [x1, y1, x2, y2] = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    cls = int(box.cls[0])
                    class_name = classes_names[cls]
                    colour = getColours(cls)

                    cv2.rectangle(frame, (x1, y1), (x2, y2), colour, 2)
                    cv2.putText(frame, f'{class_name} {box.conf[0]:.2f}',
                               (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.7, colour, 2)
                    
                    current_objects += 1
                    current_classes[class_name] += 1

        # Update statistics
        frame_count += 1
        if time.time() - start_time > 1:  # Update FPS every second
            detection_stats['fps'] = frame_count / (time.time() - start_time)
            frame_count = 0
            start_time = time.time()
        
        detection_stats['total_objects'] = current_objects
        detection_stats['class_counts'] = current_classes
        detection_stats['last_update'] = time.time()

        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')




In [ ]:
@app.route('/')
def index():
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>AI Object Detection Dashboard</title>
        <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap" rel="stylesheet">
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
        <style>
            :root {
                --primary: #4361ee;
                --secondary: #3f37c9;
                --accent: #4895ef;
                --dark: #1b263b;
                --light: #f8f9fa;
                --success: #4cc9f0;
                --danger: #f72585;
            }
            
            * {
                box-sizing: border-box;
                margin: 0;
                padding: 0;
            }
            
            body {
                font-family: 'Poppins', sans-serif;
                background-color: #f5f7fa;
                color: #333;
                line-height: 1.6;
            }
            
            .container {
                max-width: 1400px;
                margin: 0 auto;
                padding: 20px;
            }
            
            header {
                background: linear-gradient(135deg, var(--primary), var(--secondary));
                color: white;
                padding: 25px 0;
                margin-bottom: 30px;
                border-radius: 10px;
                box-shadow: 0 4px 20px rgba(67, 97, 238, 0.2);
                position: relative;
                overflow: hidden;
            }
            
            header::before {
                content: "";
                position: absolute;
                top: -50%;
                right: -50%;
                width: 100%;
                height: 200%;
                background: radial-gradient(circle, rgba(255,255,255,0.1) 1px, transparent 1px);
                background-size: 20px 20px;
                opacity: 0.3;
                transform: rotate(15deg);
            }
            
            .header-content {
                position: relative;
                z-index: 2;
                text-align: center;
                padding: 0 20px;
            }
            
            h1 {
                font-size: 2.5rem;
                font-weight: 700;
                margin-bottom: 5px;
                letter-spacing: -0.5px;
            }
            
            .subtitle {
                font-size: 1.1rem;
                opacity: 0.9;
                margin-bottom: 15px;
            }
            
            .developer {
                display: inline-block;
                background-color: rgba(255,255,255,0.2);
                padding: 5px 15px;
                border-radius: 20px;
                font-size: 0.9rem;
                margin-top: 10px;
                backdrop-filter: blur(5px);
            }
            
            .dashboard {
                display: grid;
                grid-template-columns: 2fr 1fr;
                gap: 25px;
                margin-bottom: 30px;
            }
            
            @media (max-width: 992px) {
                .dashboard {
                    grid-template-columns: 1fr;
                }
            }
            
            .card {
                background: white;
                border-radius: 12px;
                box-shadow: 0 5px 15px rgba(0,0,0,0.05);
                overflow: hidden;
                transition: transform 0.3s ease, box-shadow 0.3s ease;
            }
            
            .card:hover {
                transform: translateY(-5px);
                box-shadow: 0 10px 25px rgba(0,0,0,0.1);
            }
            
            .card-header {
                padding: 18px 25px;
                border-bottom: 1px solid rgba(0,0,0,0.05);
                display: flex;
                align-items: center;
                justify-content: space-between;
            }
            
            .card-title {
                font-size: 1.2rem;
                font-weight: 600;
                color: var(--dark);
                display: flex;
                align-items: center;
                gap: 10px;
            }
            
            .card-title i {
                color: var(--accent);
            }
            
            .video-container {
                position: relative;
                padding-bottom: 56.25%; /* 16:9 aspect ratio */
                height: 0;
                overflow: hidden;
            }
            
            .video-feed {
                position: absolute;
                top: 0;
                left: 0;
                width: 100%;
                height: 100%;
                object-fit: cover;
                border-radius: 0 0 10px 10px;
            }
            
            .stats-container {
                display: flex;
                flex-direction: column;
                gap: 20px;
            }
            
            .stat-card {
                padding: 20px;
            }
            
            .stat-row {
                display: flex;
                align-items: center;
                margin-bottom: 15px;
            }
            
            .stat-icon {
                width: 50px;
                height: 50px;
                border-radius: 12px;
                background-color: rgba(72, 149, 239, 0.1);
                display: flex;
                align-items: center;
                justify-content: center;
                margin-right: 15px;
                color: var(--accent);
                font-size: 1.2rem;
            }
            
            .stat-info {
                flex: 1;
            }
            
            .stat-label {
                font-size: 0.85rem;
                color: #6c757d;
                margin-bottom: 3px;
            }
            
            .stat-value {
                font-size: 1.5rem;
                font-weight: 600;
                color: var(--dark);
            }
            
            .class-list {
                list-style: none;
            }
            
            .class-item {
                display: flex;
                justify-content: space-between;
                align-items: center;
                padding: 12px 0;
                border-bottom: 1px solid rgba(0,0,0,0.05);
            }
            
            .class-item:last-child {
                border-bottom: none;
            }
            
            .class-name {
                display: flex;
                align-items: center;
                gap: 10px;
            }
            
            .class-color {
                width: 12px;
                height: 12px;
                border-radius: 3px;
            }
            
            .class-count {
                background-color: var(--primary);
                color: white;
                padding: 3px 10px;
                border-radius: 10px;
                font-size: 0.8rem;
                font-weight: 500;
            }
            
            footer {
                text-align: center;
                padding: 20px;
                color: #6c757d;
                font-size: 0.9rem;
            }
            
            .pulse {
                animation: pulse 1.5s infinite;
            }
            
            @keyframes pulse {
                0% { opacity: 1; }
                50% { opacity: 0.6; }
                100% { opacity: 1; }
            }
            
            .fps-indicator {
                display: inline-block;
                width: 10px;
                height: 10px;
                border-radius: 50%;
                margin-right: 8px;
            }
            
            .fps-good {
                background-color: #2ecc71;
            }
            
            .fps-warning {
                background-color: #f39c12;
            }
            
            .fps-critical {
                background-color: #e74c3c;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <header>
                <div class="header-content">
                    <h1>AI Object Detection Dashboard</h1>
                    <p class="subtitle">Real-time computer vision powered by YOLOv8</p>
                    <div class="developer">
                        <i class="fas fa-user-tie"></i> Developed by Mohammad Ali Rash
                    </div>
                </div>
            </header>
            
            <div class="dashboard">
                <div class="card">
                    <div class="card-header">
                        <h2 class="card-title">
                            <i class="fas fa-video"></i> Live Detection Feed
                        </h2>
                    </div>
                    <div class="video-container">
                        <img src="/video_feed" class="video-feed" alt="Live Video Feed">
                    </div>
                </div>
                
                <div class="stats-container">
                    <div class="card">
                        <div class="card-header">
                            <h2 class="card-title">
                                <i class="fas fa-chart-line"></i> Performance Metrics
                            </h2>
                        </div>
                        <div class="stat-card">
                            <div class="stat-row">
                                <div class="stat-icon">
                                    <i class="fas fa-tachometer-alt"></i>
                                </div>
                                <div class="stat-info">
                                    <div class="stat-label">Processing Speed</div>
                                    <div class="stat-value" id="fps">
                                        <span class="fps-indicator fps-good"></span>
                                        <span>0</span> FPS
                                    </div>
                                </div>
                            </div>
                        </div>
                    </div>
                    
                    <div class="card">
                        <div class="card-header">
                            <h2 class="card-title">
                                <i class="fas fa-object-group"></i> Detection Summary
                            </h2>
                        </div>
                        <div class="stat-card">
                            <div class="stat-row">
                                <div class="stat-icon">
                                    <i class="fas fa-boxes"></i>
                                </div>
                                <div class="stat-info">
                                    <div class="stat-label">Objects Detected</div>
                                    <div class="stat-value" id="object-count">0</div>
                                </div>
                            </div>
                            
                            <h3 style="margin: 20px 0 15px; font-size: 1rem; color: var(--dark);">
                                <i class="fas fa-list-ul" style="margin-right: 8px;"></i> Detected Classes
                            </h3>
                            
                            <ul class="class-list" id="class-list">
                                <li class="class-item">
                                    <span>No objects detected</span>
                                </li>
                            </ul>
                        </div>
                    </div>
                </div>
            </div>
            
            <footer>
                <p>AI Vision System &copy; """ + str(time.strftime("%Y")) + """ | All rights reserved</p>
            </footer>
        </div>
        
        <script>
            function updateStats() {
                fetch('/stats')
                    .then(response => response.json())
                    .then(data => {
                        // Update FPS with indicator
                        const fpsElement = document.getElementById('fps');
                        const fpsValue = data.fps.toFixed(1);
                        const indicator = fpsElement.querySelector('.fps-indicator');
                        
                        fpsElement.querySelector('span:nth-child(2)').textContent = fpsValue;
                        
                        // Change indicator color based on FPS
                        if (data.fps > 15) {
                            indicator.className = 'fps-indicator fps-good';
                        } else if (data.fps > 5) {
                            indicator.className = 'fps-indicator fps-warning';
                        } else {
                            indicator.className = 'fps-indicator fps-critical';
                        }
                        
                        // Update object count
                        document.getElementById('object-count').textContent = data.total_objects;
                        
                        // Update class list
                        const classList = document.getElementById('class-list');
                        classList.innerHTML = '';
                        
                        if (data.total_objects > 0) {
                            // Convert class counts to array and sort by count (descending)
                            const sortedClasses = Object.entries(data.class_counts)
                                .sort((a, b) => b[1] - a[1]);
                            
                            sortedClasses.forEach(([className, count]) => {
                                const item = document.createElement('li');
                                item.className = 'class-item';
                                
                                // Generate a consistent color based on class name
                                const hash = Array.from(className).reduce(
                                    (hash, char) => char.charCodeAt(0) + (hash << 5) - hash, 0);
                                const color = `hsl(${Math.abs(hash) % 360}, 70%, 50%)`;
                                
                                item.innerHTML = `
                                    <span class="class-name">
                                        <span class="class-color" style="background-color: ${color};"></span>
                                        ${className}
                                    </span>
                                    <span class="class-count">${count}</span>
                                `;
                                classList.appendChild(item);
                            });
                        } else {
                            const item = document.createElement('li');
                            item.className = 'class-item';
                            item.innerHTML = '<span>No objects detected in current frame</span>';
                            classList.appendChild(item);
                        }
                    })
                    .catch(error => console.error('Error fetching stats:', error));
                
                setTimeout(updateStats, 500);
            }
            
            // Start updating stats when page loads
            document.addEventListener('DOMContentLoaded', updateStats);
        </script>
    </body>
    </html>
    """

In [ ]:
@app.route('/stats')
def stats():
    return jsonify({
        'total_objects': detection_stats['total_objects'],
        'class_counts': detection_stats['class_counts'],
        'fps': detection_stats['fps'],
        'last_update': detection_stats['last_update']
    })

In [ ]:
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')


In [ ]:

if __name__ == '__main__':
    try:
        # Try default port first
        print("\nAttempting to start server on port 5000...")
        app.run(host='127.0.0.1', port=5000, debug=False)
    except OSError as e:
        if "Address already in use" in str(e):
            print("Port 5000 in use, trying port 8080...")
            try:
                app.run(host='127.0.0.1', port=8080, debug=False)
            except Exception as e:
                print(f"Failed to start server: {e}")
        else:
            print(f"Unexpected error: {e}")
    except Exception as e:
        print(f"Server failed to start: {e}")